# Detecting AI-Generated Images by Grok

This Jupyter Notebook is designed to classify images as either real or generated by Grok's AI tools. It uses a convolutional neural network (CNN) implemented in PyTorch for the detection.

### Author: Eduardo José Barrios García

### Import Necessary Libraries
In this section, we import all the required libraries for data processing, model construction, and evaluation.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt

### Preprocessing Function
Defines the preprocessing pipeline to convert images to grayscale, detect edges, and resize them for model input.

In [ ]:
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    resized = cv2.resize(edges, (128, 128))
    return resized

### Custom Dataset Class
A dataset class for loading and preprocessing images with corresponding labels.

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, directory, label, transform=None):
        self.directory = directory
        self.label = label
        self.transform = transform
        self.image_files = os.listdir(directory)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        filepath = os.path.join(self.directory, self.image_files[idx])
        try:
            img = cv2.imread(filepath)
            preprocessed_img = preprocess_image(img)
            img_tensor = torch.tensor(preprocessed_img, dtype=torch.float32).unsqueeze(0) / 255.0
            label_tensor = torch.tensor(self.label, dtype=torch.float32)
            return img_tensor, label_tensor
        except Exception as e:
            print(f"Error loading image {self.image_files[idx]}: {e}")
            return None, None

### Define the CNN Model
The CNN architecture is defined here, optimized for detecting the Grok logo in images.

In [ ]:
class LogoDetector(nn.Module):
    def __init__(self):
        super(LogoDetector, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

### Dataset Configuration
Load images from directories, split them into training and testing datasets, and prepare DataLoader objects.

In [ ]:
real_images_dir = "dataset/real_images"
generated_images_dir = "dataset/generated_images"

real_dataset = CustomImageDataset(real_images_dir, label=0)
generated_dataset = CustomImageDataset(generated_images_dir, label=1)

data = real_dataset + generated_dataset
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Train the Model
This function trains the model using the training dataset and evaluates the loss after each epoch.

In [ ]:
model = LogoDetector()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

train_model(model, train_loader, criterion, optimizer)

### Evaluate the Model
Evaluate the trained model on the test dataset and calculate accuracy.

In [ ]:
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            predicted = (outputs.squeeze() > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total}%")

evaluate_model(model, test_loader)

### Save and Predict
Save the trained model and use it for predicting new images.

In [ ]:
torch.save(model.state_dict(), "grok_detector_model.pth")

def predict_image(image_path, model):
    model.eval()
    img = cv2.imread(image_path)
    preprocessed_img = preprocess_image(img)
    img_tensor = torch.tensor(preprocessed_img, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0
    with torch.no_grad():
        prediction = model(img_tensor)
        return "Generated by AI (Grok)" if prediction.item() > 0.5 else "Real Image"

model.load_state_dict(torch.load("grok_detector_model.pth"))
image_path = "test_image.jpg"
prediction = predict_image(image_path, model)
print(f"Prediction for {image_path}: {prediction}")